EMOTION RECOGNITION MODEL - FER2013

DESCRIPCIÓN GENERAL
Este notebook entrena un modelo de Red Neuronal Convolucional (CNN) para reconocer emociones en imágenes faciales usando el dataset FER2013. El modelo es capaz de clasificar imágenes en 7 categorías de emociones diferentes.

FUNCIONALIDADES PRINCIPALES

1. CARGA Y PREPARACIÓN DE DATOS
- Carga de imágenes desde directorio de entrenamiento y prueba
- Aplicación de Data Augmentation en el conjunto de entrenamiento
- Normalización de imágenes (escala 0-1)
- Generadores de datos para procesamiento eficiente en lotes

2. ARQUITECTURA DEL MODELO CNN
- 4 bloques convolucionales con capas de convolución, normalización de lotes y dropout
- Pooling progresivo para reducción de dimensionalidad
- Capas densas (fully connected) para clasificación final
- Softmax para salida probabilística de 7 clases

3. ENTRENAMIENTO
- Optimizador: Adam (learning rate: 0.0001)
- Loss: Categorical Crossentropy
- Callbacks incluyen:
  - Early Stopping (paciencia: 10 epochs)
  - Reducción de tasa de aprendizaje (ReduceLROnPlateau)
  - Guardado del mejor modelo (ModelCheckpoint)

4. EMOCIONES DETECTADAS
1. Angry (Enojado)
2. Disgust (Asqueado)
3. Fear (Asustado)
4. Happy (Feliz)
5. Neutral (Neutral)
6. Sad (Triste)
7. Surprise (Sorprendido)

5. EVALUACIÓN Y VISUALIZACIÓN
- Matriz de confusión
- Reporte de clasificación (precisión, recall, f1-score)
- Gráficos de accuracy y loss durante el entrenamiento
- Función de predicción para imágenes individuales

6. OUTPUTS
- best_emotion_model.h5: Mejor modelo guardado durante el entrenamiento
- final_emotion_model.h5: Modelo final después de todos los epochs
- training_history.png: Gráficos de entrenamiento
- confusion_matrix.png: Matriz de confusión del modelo

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# Configuración de rutas - ajusta según tu estructura en Kaggle
TRAIN_DIR = '/kaggle/input/fer2013/train'
TEST_DIR = '/kaggle/input/fer2013/test'
IMG_SIZE = 48
BATCH_SIZE = 64
EPOCHS = 50

# Clases de emociones
emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
num_classes = len(emotions)

# Data Augmentation para entrenamiento
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Solo normalización para validación/test
test_datagen = ImageDataGenerator(rescale=1./255)

# Generadores de datos
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    color_mode='grayscale',
    class_mode='categorical',
    shuffle=False
)

print(f"Clases encontradas: {train_generator.class_indices}")
print(f"Imágenes de entrenamiento: {train_generator.samples}")
print(f"Imágenes de prueba: {test_generator.samples}")

# Construcción del modelo CNN
def create_emotion_cnn():
    model = keras.Sequential([
        # Bloque 1
        layers.Conv2D(64, (3, 3), activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 1), padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloque 2
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloque 3
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Bloque 4
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Capas densas
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    return model

# Crear y compilar el modelo
model = create_emotion_cnn()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-7,
        verbose=1
    ),
    ModelCheckpoint(
        'best_emotion_model.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# Entrenar el modelo
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=callbacks,
    verbose=1
)

# Guardar el modelo final
model.save('final_emotion_model.h5')
print("Modelo guardado exitosamente!")

# Visualizar resultados del entrenamiento
plt.figure(figsize=(14, 5))

# Accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Evaluación del modelo
print("\n=== Evaluación en el conjunto de prueba ===")
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

# Predicciones para la matriz de confusión
test_generator.reset()
predictions = model.predict(test_generator, steps=len(test_generator))
y_pred = np.argmax(predictions, axis=1)
y_true = test_generator.classes

# Reporte de clasificación
print("\n=== Reporte de Clasificación ===")
print(classification_report(y_true, y_pred, target_names=emotions))

# Matriz de confusión
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=emotions, yticklabels=emotions)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Función para predecir emociones en imágenes individuales
def predict_emotion(image_path):
    """
    Predice la emoción de una imagen individual
    """
    img = keras.preprocessing.image.load_img(
        image_path, 
        target_size=(IMG_SIZE, IMG_SIZE),
        color_mode='grayscale'
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    prediction = model.predict(img_array, verbose=0)
    emotion_idx = np.argmax(prediction)
    confidence = prediction[0][emotion_idx]
    
    return emotions[emotion_idx], confidence

# Ejemplo de uso de la función de predicción
# emotion, conf = predict_emotion('path/to/image.jpg')
# print(f"Emoción detectada: {emotion} (Confianza: {conf:.2%})")

print("\n¡Entrenamiento completado exitosamente!")

: 